<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #8: Modelado de temas
`Fecha de entrega: 🎃 Octubre 31, 2020. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [48]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

import pyLDAvis.gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [53]:
!pip install pyLDAvis

In [58]:
# Cargar datos
data = pd.read_csv('D:/NLP/Clase10-24-10-2020/reviews_vidjew_es.csv')
print(f"Tenemos {data.shape[0]:,d}documentos.")
data.head()

Tenemos 1,000documentos.


,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


###  `[12 pts]` Punto 1: Hacer pre-procesamiento del texto

In [59]:
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
stopwords_sp = stopwords.words('spanish')

In [60]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords_sp]
    return texto

In [63]:
data['PP'] = data['review_body'].apply(lambda texto: pre_procesado(texto))

data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,pp,PP
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]","[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw...","[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl...","[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble...","[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg...","[cable, funciona, bien, podria, ser, mas, larg..."


###  `[13 pts]` Punto 2: Modelo de LDA

In [64]:
import pyLDAvis.gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from pprint import pprint

In [79]:
# Crear una representación de los documentos en forma de diccionario
dictionary = Dictionary(data['PP'].values)

# Filtrar palabras muy frecuentes o infrecuentes

dictionary.filter_extremes(no_below=5, no_above=0.5)
 
corpus = [dictionary.doc2bow(text) for text in data['PP'].values]

#Train the topic model

model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=7, passes=10)

dictionary = Dictionary(data['PP'].values)

In [80]:
# Paso 4: Resultados

model.print_topics(num_words=10)

[(0,
  '0.047*"bien" + 0.031*"pendientes" + 0.021*"llegado" + 0.018*"producto" + 0.018*"grandes" + 0.017*"pequeños" + 0.015*"recibido" + 0.014*"si" + 0.014*"plata" + 0.014*"bonitos"'),
 (1,
  '0.023*"bien" + 0.022*"producto" + 0.021*"calidad" + 0.019*"caja" + 0.017*"bastante" + 0.015*"juego" + 0.015*"mal" + 0.015*"mas" + 0.014*"verdad" + 0.014*"buena"'),
 (2,
  '0.048*"juego" + 0.023*"bien" + 0.022*"pedido" + 0.019*"día" + 0.019*"amazon" + 0.017*"problema" + 0.016*"si" + 0.013*"ahora" + 0.013*"así" + 0.012*"jugar"'),
 (3,
  '0.042*"buen" + 0.033*"perfecto" + 0.032*"producto" + 0.021*"precio" + 0.018*"si" + 0.018*"llegó" + 0.015*"bien" + 0.015*"juego" + 0.013*"correcto" + 0.013*"tiempo"'),
 (4,
  '0.032*"bien" + 0.028*"bonito" + 0.019*"calidad" + 0.019*"si" + 0.018*"foto" + 0.016*"cadena" + 0.015*"colgante" + 0.014*"pequeño" + 0.014*"queda" + 0.013*"juego"'),
 (5,
  '0.067*"calidad" + 0.048*"precio" + 0.029*"buena" + 0.023*"bien" + 0.021*"puede" + 0.018*"ps" + 0.017*"mando" + 0.014*"ped

###  `[25 pts]` Punto 3: Visualización de LDA

In [81]:
import pyLDAvis.gensim

In [82]:
lda_display = pyLDAvis.gensim.prepare(model, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)

IndexError: index 505 is out of bounds for axis 1 with size 505